In [ ]:
import shutil
shutil.unpack_archive("/content/cemri new.zip", "/content/cemri/")

In [ ]:
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img

In [ ]:
#loading images using load_img

#converting images into array

#putting array images into train_imgs

IMG_DIM = (256, 256)
train_files = glob.glob('/content/cemri/trainhis/*')
train_imgs = [img_to_array(load_img(img, target_size=IMG_DIM)) for img in train_files]
train_imgs = np.array(train_imgs)

#getting labels from the image name
train_labels = [fn.split('/')[-1].split('_')[0]  for fn  in train_files]

validation_files = glob.glob('/content/cemri/testhis/*')
validation_imgs = [img_to_array(load_img(img, target_size=IMG_DIM)) for img in validation_files]
validation_imgs = np.array(validation_imgs)
validation_labels = [fn.split('/')[-1].split('_')[0]  for fn in validation_files]



In [ ]:
print(len(validation_labels),len(train_labels))

print(validation_imgs.shape)

614 2450
(614, 256, 256, 3)


In [ ]:
#normalizing individual channel in images array

train_imgs_scaled = train_imgs.astype('float32')
validation_imgs_scaled  = validation_imgs.astype('float32')
train_imgs_scaled /= 255.
validation_imgs_scaled /= 255.



In [ ]:
batch_size = 64
epochs = 6
input_shape = (256,256, 3)

In [ ]:
#encoding image labels 

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(train_labels)
train_labels_enc = le.transform(train_labels)
validation_labels_enc = le.transform(validation_labels)

#tranforming labels to catogerical
from keras.utils import to_categorical
train_labels_c = to_categorical(train_labels_enc)
validation_labels_c = to_categorical(validation_labels_enc)

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.models import Sequential
from keras import optimizers
import tensorflow as tf

model = Sequential()

model.add(Conv2D(16, kernel_size=(3, 3), activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512, activation=tf.nn.relu))
model.add(Dense(3, activation=tf.nn.softmax))

opt = optimizers.Adam(learning_rate = 0.001) 
model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 254, 254, 16)      448       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 127, 127, 16)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 125, 125, 64)      9280      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 62, 62, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 60, 60, 128)       73856     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 28, 28, 256)      

In [ ]:
history = model.fit(x=train_imgs_scaled, y=train_labels_c,
                    validation_data=(validation_imgs_scaled, validation_labels_c),
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1)

Train on 2450 samples, validate on 614 samples
Epoch 1/6
2450/2450 [==============================] - 294s 120ms/step - loss: 0.5745 - accuracy: 0.7329 - val_loss: 0.3367 - val_accuracy: 0.8415
Epoch 2/6
2450/2450 [==============================] - 293s 120ms/step - loss: 0.2781 - accuracy: 0.8754 - val_loss: 0.2507 - val_accuracy: 0.8974
Epoch 3/6
2450/2450 [==============================] - 294s 120ms/step - loss: 0.2046 - accuracy: 0.9103 - val_loss: 0.1807 - val_accuracy: 0.9240
Epoch 4/6
2450/2450 [==============================] - 293s 120ms/step - loss: 0.1575 - accuracy: 0.9358 - val_loss: 0.1874 - val_accuracy: 0.9316
Epoch 5/6
2450/2450 [==============================] - 293s 120ms/step - loss: 0.1341 - accuracy: 0.9452 - val_loss: 0.1630 - val_accuracy: 0.9397
Epoch 6/6
2450/2450 [==============================] - 292s 119ms/step - loss: 0.1015 - accuracy: 0.9604 - val_loss: 0.2173 - val_accuracy: 0.9115


In [ ]:
from keras.models import Sequential, Model

In [ ]:
model_feat = Model(inputs=model.input,outputs=model.get_layer('max_pooling2d_8').output)

feat_train = model_feat.predict(train_imgs_scaled)
print(feat_train.shape)

feat_val = model_feat.predict(validation_imgs_scaled)
print(feat_val.shape)


(2450, 14, 14, 256)
(614, 14, 14, 256)


In [ ]:
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
feat_train[0][0].shape

(14, 256)

In [ ]:
from sklearn.svm import SVC

svm = SVC(kernel='rbf')

svm.fit(feat_train,np.argmax(train_labels_c,axis=1))

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
svm.score(feat_train,np.argmax(train_labels_c,axis=1))

0.9514285714285714

In [ ]:
svm.score(feat_val,np.argmax(validation_labels_c,axis=1))


0.9315960912052117

In [ ]:
rm -r '/content/cemri/train'